<a href="https://colab.research.google.com/github/AugustoCRX/bootcamp_blue/blob/main/Arquivo_principal_V1_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import random
import numpy as np
import pandas as pd
import re

def export_dataset():
    #Lista de links (arquivos estão dentro do Google Sheets)
    #Foi necessário criar vários arquivos arquivos por conta do tamanho dos arquivos e o google sheets estava apresentando problemas de memória
    test_links = {
        'link_01': 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQvClyfCtk8JD82zXYSipnN7iCJ0XG0V-z6tcII_14W7tQYPpLLOK45kKWLQf8TBfyRNhH9mlEXiowR/pub?output=csv',
        'link_02': 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQq0fLn7Dkg0-Lg2i9FUD0iL-xLVHn1Gtorx8wLlA7flMftChIJf_PXgziaHshCnxlX52bZJZ2EKFsu/pub?output=csv'
    }
    train_links ={
        'link_01':'https://docs.google.com/spreadsheets/d/e/2PACX-1vS2E4FyELPrgf1rPIeyh4lzQTOtftYYK6vyr7m5TN7cYDZTo4tLy1jZQ06mnnwo0NrJ5ZR5IOm2-ndN/pub?output=csv',
        'link_02':'https://docs.google.com/spreadsheets/d/e/2PACX-1vT-NHTDb71tp400NvyVObenotd8uUATgPgl4nlGwvqpwTTMoCmc8bH7YmyvmzBg_TcIAxoNkC0upfoe/pub?output=csv',       
        'link_03':'https://docs.google.com/spreadsheets/d/e/2PACX-1vQux2yo8RJg5YdLAF7t7iOv1e22RLqp7INPPrQkqDdJVzkbkdJl2QQGiW_KWtnw8mxhbxXu6wgV9N-2/pub?output=csv',
        'link_04':'https://docs.google.com/spreadsheets/d/e/2PACX-1vTFoYy6F6IuVXBhhGazTbuu5LUqItSrOYh88SPj41JjAtKckTyrtygOH7ePKIKJfN80o_7sNPip5UaG/pub?output=csv',
        'link_05':'https://docs.google.com/spreadsheets/d/e/2PACX-1vTooCcyaoXnkazJ6HI95oc-Ll45cMY4qnjh6iIa9HVJfhTej2z9EBDVVHgplEhqLfegmeucud1mXMhS/pub?output=csv'
    }
   
    #Download e leitura do arquivo de treinamento
    temp_df_test = pd.DataFrame()
    for i in test_links:
        file_temp = pd.read_csv(test_links[i])
        temp_df_test = pd.concat([temp_df_test,file_temp]) 
    temp_df_test.reset_index(drop=True, inplace=True)
    temp_df_test.drop('Unnamed: 0',axis=1, inplace= True)

    #Download e leitura do arquivo de teste
    temp_df_train = pd.DataFrame()
    for i in train_links:
        file_temp = pd.read_csv(train_links[i])
        temp_df_train = pd.concat([temp_df_train,file_temp]) 
    temp_df_train.reset_index(drop=True, inplace=True)
    temp_df_train.drop('Unnamed: 0',axis=1, inplace= True)    
    
    return temp_df_test, temp_df_train

test, train = export_dataset()

| Nome da coluna | Descrição |
| :----: | :----: |
| test_id | Numero identificador dos itens da lista |
| name | Titulo da lista*|
| item_condition_id | Numero da condição do item, varia de 1 a 5, sendo 1 a pior e 5 a melhor |
| category_name | Categorias da lista |
| brand_name | Nome da marca do item (existe nulos) |
| price | Preço do produto |
| shipping | Taxa de envio, pago por: 1 - pelo vendedor 0 - pelo comprador |
| item_description | Descrição completa do item* |
| date | Data |
| stock | Estoque |

*¹- Nessas colunas existe um valor chamado [rm], esse valor significa "removed" ou removido, significa que o preço (exemplo: $20) foi removido.

In [40]:
#o rm também foi adicionado em descrições completas, visualizar arquivo fixado no discord

In [41]:
#existem valores repitidos na coluna brand_name (exemplo: adidas e adidas NEO, as duas são a mesma coisa)
#necessário tratar esses dados

In [42]:
train_vis = train[0:1000]
test_vis = test[0:1000]

In [43]:
test1 = train.query("name == '[rm]'")

In [44]:
test2 = train.query("item_description == '[rm]'")

In [45]:
train['item_condition_id'].value_counts()

1    640549
3    432161
2    375479
4     31962
5      2384
Name: item_condition_id, dtype: int64

In [46]:
#category_name
#lista de categorias do nome da lista
#divido por caracteres especiais
#necessário limpeza para avaliação, talvez dividir em lista ou dummmies?

In [47]:
train.shape

(1482535, 10)

In [48]:
test_array = train['brand_name'].drop_duplicates().unique()
train['brand_name'].unique().shape

(4810,)

In [49]:
train.info()    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482535 entries, 0 to 1482534
Data columns (total 10 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   train_id           1482535 non-null  int64 
 1   name               1482535 non-null  object
 2   item_condition_id  1482535 non-null  int64 
 3   category_name      1476208 non-null  object
 4   brand_name         849853 non-null   object
 5   price              1482535 non-null  object
 6   shipping           1482535 non-null  int64 
 7   item_description   1482531 non-null  object
 8   date               1482535 non-null  object
 9   stock              1482535 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 113.1+ MB


In [50]:
# Remove special caracters from a list and return itself
def remove_especial_caracters(dataframe_column):
    complete_array = []
    for n in range(len(dataframe_column)):
        array = str(dataframe_column[n]).split('/')
        for i in range(len(array)):
            x = str(array[i])
            complete_array.append(x)
    return complete_array

train_frame = remove_especial_caracters(train['category_name'])
test_frame = remove_especial_caracters(test['category_name'])

unique, counts = np.unique(test_frame, return_counts=True)
frame = pd.DataFrame(np.stack((unique, counts), axis = 1), columns= ['Category name', 'Count'])

In [51]:
#puttting in order
frame['Count'] = pd.to_numeric(frame['Count'])

frame.sort_values(by=['Count'], inplace=True, ascending = False)

In [52]:
frame.loc[frame['Count']>5000]

,Category name,Count
898,Women,319042
80,Beauty,97265
463,Kids,80299
34,Athletic Apparel,62669
721,Shoes,61731
...,...,...
251,Daily & Travel items,5406
575,One-Pieces,5390
684,Rings,5121
731,Skirts,5029


In [53]:
#looking all categories names
print(frame['Category name'].unique())
print(frame['Category name'].describe())
display (frame[frame['Category name'].isnull()])

['Women' 'Beauty' 'Kids' 'Athletic Apparel' 'Shoes' 'Electronics' 'Makeup'
 'Other' 'Tops & Blouses' 'Men' 'Home' 'Jewelry' 'Pants, Tights, Leggings'
 'Toys' 'Dresses' 'Cell Phones & Accessories' 'Face' 'T-Shirts'
 'Vintage & Collectibles' "Women's Handbags" "Women's Accessories" 'Jeans'
 'Video Games & Consoles' 'Sweaters' 'Underwear' 'Lips' 'Games' 'Handmade'
 'Skin Care' 'Athletic' 'Eyes' 'Sports & Outdoors' 'Home Décor'
 'Coats & Jackets' 'Fragrance' 'Cases, Covers & Skins' 'Shorts'
 'Kitchen & Dining' 'Bras' 'Tops' 'Tank, Cami' 'Swimwear' 'Blouse'
 'Above Knee, Mini' 'Boots' 'Necklaces' 'Makeup Palettes'
 "Men's Accessories" 'Girls 2T-5T' 'Accessories' 'Girls 0-24 Mos'
 'Shirts & Tops' 'Girls (4+)' 'Shoulder Bag' 'Fashion Sneakers' 'T-shirts'
 'Sandals' 'Boys 0-24 Mos' 'Tops & T-Shirts' 'Wallets' 'Boys (4+)'
 'Knee-Length' 'Boys 2T-5T' 'Bracelets' 'Dolls & Accessories'
 'Slim, Skinny' 'Home Décor Accents' 'Tools & Accessories' 'Two-Piece'
 'Action Figures & Statues' 'Hats' 'Media'

,Category name,Count


In [54]:
#looking for general categories
frame[frame['Category name']=="Other"]

,Category name,Count
579,Other,54003


In [55]:
#looking for general categories
frame[frame['Category name']=="Others"]

,Category name,Count
582,Others,654


##Limpeza de dados

In [56]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482535 entries, 0 to 1482534
Data columns (total 10 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   train_id           1482535 non-null  int64 
 1   name               1482535 non-null  object
 2   item_condition_id  1482535 non-null  int64 
 3   category_name      1476208 non-null  object
 4   brand_name         849853 non-null   object
 5   price              1482535 non-null  object
 6   shipping           1482535 non-null  int64 
 7   item_description   1482531 non-null  object
 8   date               1482535 non-null  object
 9   stock              1482535 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 113.1+ MB


In [57]:
# looking the categories name in original dataset
print(train['category_name'].unique())
print(train['category_name'].describe())
display (train[train['category_name'].isnull()])

['Men/Tops/T-shirts' 'Electronics/Computers & Tablets/Components & Parts'
 'Women/Tops & Blouses/Blouse' ... 'Handmade/Jewelry/Clothing'
 'Vintage & Collectibles/Supplies/Ephemera' 'Handmade/Pets/Blanket']
count                                            1476208
unique                                              1287
top       Women/Athletic Apparel/Pants, Tights, Leggings
freq                                               60177
Name: category_name, dtype: object


,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,date,stock
122,122,Bundle,3,NaN,NaN,59,0,Sizes and info of clothes can be found on thei...,17-4-2018,25
155,155,3 Nora Roberts Books,3,NaN,NaN,10,1,For aferg16.,16-1-2018,2
258,258,ACER Laptop,5,NaN,NaN,14,0,"Powers on, no screen display, no external dama...",19-8-2018,1
984,984,AUTHENTIC BRWN MICHAEL KORS MAKEUP STAIN,4,NaN,Michael Kors,18,0,No description yet,5-2-2018,6
1185,1185,Teenage Mutant Ninja Turtle Van/ Extras,3,NaN,Nickelodeon,9,0,"Teenage Mutant Ninja Turtle Van, April O'Neil ...",13-4-2018,9
...,...,...,...,...,...,...,...,...,...,...
1481249,1481249,Only For Erin. Do Not Buy,3,NaN,NaN,31,0,No description yet,20-4-2018,13
1481750,1481750,American Eagle Jeggings,2,NaN,American Eagle,17,0,Worn a couple of time Size 4,12-10-2018,5
1482252,1482252,Bundle For All Honey :),3,NaN,NaN,40,0,No description yet,6-2-2018,1
1482443,1482443,HOLD 14g Purple Prong Nipple Bars Rings,1,NaN,NaN,22,1,14g Purple Prong Set Nipple Bars Rings [surgic...,20-3-2018,17


In [58]:
# droping null categories names and brands names(around 3000 values)
dropnull = train[train.category_name.isnull() & train.brand_name.isnull()].index
train = train.drop(dropnull, axis=0).reset_index()

In [59]:
# cheking the categories name
print(train['category_name'].unique())
print(train['category_name'].describe())
display (train[train['category_name'].isnull()])

['Men/Tops/T-shirts' 'Electronics/Computers & Tablets/Components & Parts'
 'Women/Tops & Blouses/Blouse' ... 'Handmade/Jewelry/Clothing'
 'Vintage & Collectibles/Supplies/Ephemera' 'Handmade/Pets/Blanket']
count                                            1476208
unique                                              1287
top       Women/Athletic Apparel/Pants, Tights, Leggings
freq                                               60177
Name: category_name, dtype: object


,index,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,date,stock
981,984,984,AUTHENTIC BRWN MICHAEL KORS MAKEUP STAIN,4,NaN,Michael Kors,18,0,No description yet,5-2-2018,6
1182,1185,1185,Teenage Mutant Ninja Turtle Van/ Extras,3,NaN,Nickelodeon,9,0,"Teenage Mutant Ninja Turtle Van, April O'Neil ...",13-4-2018,9
1271,1274,1274,Black Ribbed Off-the-Shoulder Crop Top,2,NaN,Urban Outfitters,15,0,From the silence + noise brand sold at Urban O...,29-2-2018,13
1317,1320,1320,Nice Condition Pet Escort,2,NaN,PetSafe,12,0,Nice condition I will wipe down before selling,10-2-2018,12
1733,1739,1739,SchoolWear Uniform Pants Bugle Boy,2,NaN,Bugle Boy,5,1,Selling Kids Pant celebrate the back to school...,8-8-2018,3
...,...,...,...,...,...,...,...,...,...,...,...
1475719,1479167,1479167,Steve Madden Black Heels,3,NaN,Steve Madden,24,0,Please look at the photos before purchasing,5-2-2018,2
1476649,1480098,1480098,Nike Slides *On hold*,3,NaN,Nike,14,0,Worn but not a lot,8-4-2018,1
1476775,1480224,1480224,2xl Amelia Lularoe,2,NaN,LuLaRoe,50,1,New without tags. Only tried on. Only the dres...,17-8-2018,32
1478295,1481750,1481750,American Eagle Jeggings,2,NaN,American Eagle,17,0,Worn a couple of time Size 4,12-10-2018,5


Antes tinhamos 6327 categorias nulas, agora temos 2870. As outras irei substituir por "Other" que já temos outras 54003 vezes utilizados no dataset.

In [60]:
# replace null values to "Other" 
train['category_name'] = train['category_name'].fillna("Other")

In [61]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1479078 entries, 0 to 1479077
Data columns (total 11 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   index              1479078 non-null  int64 
 1   train_id           1479078 non-null  int64 
 2   name               1479078 non-null  object
 3   item_condition_id  1479078 non-null  int64 
 4   category_name      1479078 non-null  object
 5   brand_name         849853 non-null   object
 6   price              1479078 non-null  object
 7   shipping           1479078 non-null  int64 
 8   item_description   1479074 non-null  object
 9   date               1479078 non-null  object
 10  stock              1479078 non-null  int64 
dtypes: int64(5), object(6)
memory usage: 124.1+ MB


In [62]:
# looking the item_description in original dataset
print(train['item_description'].unique())
print(train['item_description'].describe())
display (train[train['item_description'].isnull()])

['No description yet'
 'This keyboard is in great condition and works like it came out of the box. All of the ports are tested and work perfectly. The lights are customizable via the Razer Synapse app on your PC.'
 'Adorable top with a hint of lace and a key hole in the back! The pale pink is a 1X, and I also have a 3X available in white!'
 ... 'Used once or twice, still in great shape.'
 'There is 2 of each one that you see! So 2 red 2 orange and 2 of the big red and orange ones! They are from world market!'
 'New with tag, red with sparkle. Firm price, no free shipping.']
count                1479074
unique               1278602
top       No description yet
freq                   82061
Name: item_description, dtype: object


,index,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,date,stock
510332,511535,511535,Shoes for Michelle,4,Kids/Girls 0-24 Mos/Shoes,NaN,9,0,NaN,5-11-2018,6
859172,861230,861230,Lipgloss,4,Beauty/Makeup/Lips,NaN,49,0,NaN,26-2-2018,1
1222071,1224924,1224924,Disney Minnie Head band,3,Women/Women's Accessories/Hair Accessories,Disney,9,0,NaN,18-9-2018,5
1261296,1264242,1264242,For Bianca,3,Women/Women's Accessories/Scarves & Wraps,NaN,10,1,NaN,4-8-2018,7


In [63]:
#have just 4 lines of null item description, i will remove them
train = train.dropna(subset = ['item_description']).reset_index()
train = train.drop(columns = ['level_0','index','train_id','shipping'])

Sobre o Shipping acredito que não vale a pena manter, ele mostra somente por quem a taxa de envio foi paga, não intefere no preço.

In [64]:
# looking the brands name in original dataset
print(train['brand_name'].unique())
print(train['brand_name'].describe())
display (train[train['brand_name'].isnull()])

[nan 'Razer' 'Target' ... 'Astroglide' 'Cumberland Bay' 'Kids Only']
count     849852
unique      4809
top         PINK
freq       54088
Name: brand_name, dtype: object


,name,item_condition_id,category_name,brand_name,price,item_description,date,stock
0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10,No description yet,17-5-2018,27
3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35,New with tags. Leather horses. Retail for [rm]...,19-2-2018,1
4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44,Complete with certificate of authenticity,16-4-2018,13
5,Bundled items requested for Ruie,3,Women/Other/Other,NaN,59,"Banana republic bottoms, Candies skirt with ma...",8-11-2018,6
9,Porcelain clown doll checker pants VTG,3,Vintage & Collectibles/Collectibles/Doll,NaN,8,I realized his pants are on backwards after th...,20-6-2018,1
...,...,...,...,...,...,...,...,...
1479065,Harry Potter Shirt! Women M/ Girl XL,2,Women/Tops & Blouses/T-Shirts,NaN,12,"Great Harry Potter Shirt! ""Hogwarts, School of...",24-12-2018,5
1479066,Blk/white ribbed mock neck bodysuit M,1,Women/Tops & Blouses/Blouse,NaN,10,Brand new black and white ribbed mock neck bod...,15-9-2018,1
1479071,21 day fix containers and eating plan,2,Sports & Outdoors/Exercise/Fitness accessories,NaN,12,"Used once or twice, still in great shape.",6-8-2018,15
1479072,World markets lanterns,3,Home/Home Décor/Home Décor Accents,NaN,45,There is 2 of each one that you see! So 2 red ...,12-2-2018,20


In [65]:
#code looking for any word or sentence in dataset
train[train['name'].str.find('Harry Potter')!=-1]

,name,item_condition_id,category_name,brand_name,price,item_description,date,stock
235,Lego Harry Potter Years 1-4 Wii,2,Electronics/Video Games & Consoles/Games,Nintendo,19,Lego Harry Potter Years 1-4 Wii E10+ Warner Br...,7-1-2018,8
756,Harry Potter PC game free shipping,3,Electronics/Video Games & Consoles/Games,Microsoft,6,Works great Fast shipping with tracking Weighs...,11-5-2018,9
3747,Lularoe TC Harry Potter Owls Blue,1,"Women/Athletic Apparel/Pants, Tights, Leggings",NaN,60,Brand new. Harry Potter Owl. Extremely hard to...,17-2-2018,18
4200,Harry Potter wands,2,Vintage & Collectibles/Collectibles/Souvenir,NaN,50,Like new. No flaws. Snape and mad eye,12-11-2018,9
5058,Nintendo Wii game Harry Potter,2,Vintage & Collectibles/Electronics/Video Game,Nintendo,11,Like new,19-1-2018,1
...,...,...,...,...,...,...,...,...
1476435,Harry Potter - Gryffindor Sport Shirt,1,Women/Tops & Blouses/T-Shirts,NaN,14,Size small. Excellent condition; bought from U...,3-10-2018,37
1477261,Harry Potter Books Case - Snap On,2,"Electronics/Cell Phones & Accessories/Cases, C...",NaN,6,FREE SHIPPING! This iPhone 6 case is a snap on...,21-10-2018,1
1477329,5 Harry Potter Bertie Botts,1,Other/Books/Literature & Fiction,NaN,19,Sealed. left over from HP party. Exp feb 2018,7-7-2018,6
1478782,FREESHIPPING! Harry Potter Mystery Mini,2,Kids/Toys/Action Figures & Statues,Funko,12,"Harry Potter Mystery Mini, Barnes & Noble Excl...",26-10-2018,1


In [66]:
#I will raplace all 629225 null values in "train['brand_name']" for other again, after we can looking again
train['brand_name'] = train['brand_name'].fillna("Other")

In [67]:
#we can also remove all this null values, but we lost 629225 lines

#train['brand_name'] = train['brand_name'].dropna() 

In [68]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1479074 entries, 0 to 1479073
Data columns (total 8 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   name               1479074 non-null  object
 1   item_condition_id  1479074 non-null  int64 
 2   category_name      1479074 non-null  object
 3   brand_name         1479074 non-null  object
 4   price              1479074 non-null  object
 5   item_description   1479074 non-null  object
 6   date               1479074 non-null  object
 7   stock              1479074 non-null  int64 
dtypes: int64(2), object(6)
memory usage: 90.3+ MB


In [69]:
# looking the dates in original dataset
print(train['date'].unique())
print(train['date'].describe())
display (train[train['date'].isnull()])

['17-5-2018' '17-3-2018' '24-10-2018' '19-2-2018' '16-4-2018' '8-11-2018'
 '29-1-2018' '25-7-2018' '12-4-2018' '20-6-2018' '20-8-2018' '4-9-2018'
 '29-11-2018' '18-1-2018' '1-2-2018' '28-4-2018' '11-10-2018' '10-7-2018'
 '13-1-2018' '17-10-2018' '5-3-2018' '12-1-2018' '15-9-2018' '25-2-2018'
 '18-2-2018' '23-2-2018' '5-9-2018' '28-5-2018' '13-3-2018' '8-7-2018'
 '6-2-2018' '14-2-2018' '19-10-2018' '23-6-2018' '11-5-2018' '1-8-2018'
 '10-4-2018' '3-5-2018' '26-8-2018' '8-6-2018' '27-6-2018' '15-6-2018'
 '12-5-2018' '17-6-2018' '15-2-2018' '13-11-2018' '1-6-2018' '2-2-2018'
 '4-5-2018' '29-6-2018' '14-5-2018' '15-5-2018' '22-10-2018' '21-5-2018'
 '19-11-2018' '18-8-2018' '19-7-2018' '25-6-2018' '24-11-2018'
 '13-10-2018' '21-10-2018' '2-11-2018' '23-11-2018' '29-4-2018'
 '20-2-2018' '18-11-2018' '6-4-2018' '3-3-2018' '7-2-2018' '29-10-2018'
 '14-9-2018' '21-6-2018' '13-8-2018' '11-7-2018' '19-3-2018' '18-10-2018'
 '3-1-2018' '23-7-2018' '9-8-2018' '8-5-2018' '10-8-2018' '14-6-2018'
 '24-

,name,item_condition_id,category_name,brand_name,price,item_description,date,stock


In [91]:
#Creating new dates columns
train['date2'] = pd.to_datetime(train['date'], errors='coerce')    #corrige os erros com os valores das datas de nascimento, adequa para DateTime
train['day'] = train['date2'].dt.day
train['month'] = train['date2'].dt.month
train['year'] = train['date2'].dt.year

In [80]:
# looking the null dates
display (train[train['date2'].isnull()])

,name,item_condition_id,category_name,brand_name,price,item_description,date,stock,date2,day,month,year
348,LuLaRoe Halloween Frankenstein Leggings,2,"Women/Athletic Apparel/Pants, Tights, Leggings",Independent,24,LuLaRoe Leggings One Size Halloween Frankenste...,29-2-2018,5,NaT,NaN,NaN,NaN
991,New iPhone 6/6s Purple Ombre Soft Case,1,"Electronics/Cell Phones & Accessories/Cases, C...",Other,10,PRICES ARE FIRM UNLESS YOU BUNDLE! ALL OFFERS ...,29-2-2018,1,NaT,NaN,NaN,NaN
1166,Rambler YETI Coffee Cups 2 PC,1,Vintage & Collectibles/Serving/Mug,Other,40,DON'T purchases before reading following detai...,29-2-2018,10,NaT,NaN,NaN,NaN
1271,Black Ribbed Off-the-Shoulder Crop Top,2,Other,Urban Outfitters,15,From the silence + noise brand sold at Urban O...,29-2-2018,13,NaT,NaN,NaN,NaN
1304,Active leggings,2,"Women/Athletic Apparel/Pants, Tights, Leggings",Aeropostale,21,There super cute but there a little see throug...,29-2-2018,8,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1478089,BUNDLE FOR SYLVIA HERNANDEZ,2,Women/Women's Handbags/Shoulder Bag,Burberry,75,"BURBERRY SHOULDER BAG, IN EXCELLENT CONDITION....",29-2-2018,1,NaT,NaN,NaN,NaN
1478453,Kat von d shade +light eyeshadow palette,3,Beauty/Makeup/Makeup Palettes,Kat Von D,19,Bought on here awhile ago one color was broken...,29-2-2018,18,NaT,NaN,NaN,NaN
1478458,Vans authentic,1,Women/Shoes/Fashion Sneakers,VANS,34,"Red authentic vans Size men 6, women 7.5",29-2-2018,3,NaT,NaN,NaN,NaN
1478465,Dr martens women pink shoes,2,Women/Shoes/Athletic,Dr. Martens,41,I also have a man size,29-2-2018,1,NaT,NaN,NaN,NaN


In [75]:
test_date = train[train['date2'].isnull()]
test_date['date'].describe()

Todas as datas que ficaram nulas são do dia 29/02/2018. Como esse dia não existe vamos substituí-lo pelo dia 28/02/2018

In [89]:
train['date'] =  train['date'].replace('29-2-2018','28-2-2018')

In [ ]:
#refazendo as colunas para não ter valores nulos
train['date2'] = pd.to_datetime(train['date'], errors='coerce')    #corrige os erros com os valores das datas de nascimento, adequa para DateTime
train['day'] = train['date2'].dt.day
train['month'] = train['date2'].dt.month
train['year'] = train['date2'].dt.year

In [93]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1479074 entries, 0 to 1479073
Data columns (total 12 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   name               1479074 non-null  object        
 1   item_condition_id  1479074 non-null  int64         
 2   category_name      1479074 non-null  object        
 3   brand_name         1479074 non-null  object        
 4   price              1479074 non-null  object        
 5   item_description   1479074 non-null  object        
 6   date               1479074 non-null  object        
 7   stock              1479074 non-null  int64         
 8   date2              1479074 non-null  datetime64[ns]
 9   day                1479074 non-null  int64         
 10  month              1479074 non-null  int64         
 11  year               1479074 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(6)
memory usage: 135.4+ MB


In [96]:
train.head()

,name,item_condition_id,category_name,brand_name,price,item_description,date,stock,date2,day,month,year
0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,Other,10,No description yet,17-5-2018,27,2018-05-17,17,5,2018
1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52,This keyboard is in great condition and works ...,17-3-2018,15,2018-03-17,17,3,2018
2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10,Adorable top with a hint of lace and a key hol...,24-10-2018,14,2018-10-24,24,10,2018
3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,Other,35,New with tags. Leather horses. Retail for [rm]...,19-2-2018,1,2018-02-19,19,2,2018
4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,Other,44,Complete with certificate of authenticity,16-4-2018,13,2018-04-16,16,4,2018
